In [1]:
from os import path

import torch
from detectron2.checkpoint import DetectionCheckpointer
from detectron2.config import LazyConfig, instantiate
from diffusers import AutoPipelineForText2Image, DiffusionPipeline
from diffusers_interpret import StableDiffusionPipelineDetExplainer

from gill import layers

from torch.cuda import amp

In [2]:
model_path = '/mnt/workspace/model'
eva_id = 'zacbi2023/eva02'
sd_id = 'AI-ModelScope/stable-diffusion-v1-5'

In [3]:
# prepare eva
eva_base_path = path.join(model_path, eva_id)
eva_coco_config_rpath = 'projects/ViTDet/configs/eva2_o365_to_coco/eva2_o365_to_coco_cascade_mask_rcnn_vitdet_l_8attn_1536_lrd0p8.py'
eva_config_path = path.join(eva_base_path, eva_coco_config_rpath)

# replace with your eva02 weights path
eva_coco_weights_rpth = 'checkpoints/eva02_L_coco_seg_sys_o365.pth'
eva_weights_path = path.join(eva_base_path, eva_coco_weights_rpth)

custum_cfg = ['MODEL.RETINANET.SCORE_THRESH_TEST', 0.5,
                'MODEL.ROI_HEADS.SCORE_THRESH_TEST', 0.5,
                'MODEL.PANOPTIC_FPN.COMBINE.INSTANCES_CONFIDENCE_THRESH', 0.5,
                'DATASETS.TEST', [],
                'MODEL.WEIGHTS', eva_weights_path]
eva_cfg = LazyConfig.load(eva_config_path)
LazyConfig.apply_overrides(
    eva_cfg, [f"{key}={value}" for key, value in zip(custum_cfg[::2], custum_cfg[1::2])])

device = 'cuda'
eva = instantiate(eva_cfg.model).to(device)
DetectionCheckpointer(eva).load(eva_weights_path)
eva.eval()

======== shape of rope freq torch.Size([256, 64]) ========
======== shape of rope freq torch.Size([9216, 64]) ========


The checkpoint state_dict contains keys that are not used by the model:
  pixel_mean
  pixel_std


GeneralizedRCNN(
  (backbone): SimpleFeaturePyramid(
    (simfp_2): Sequential(
      (0): ConvTranspose2d(1024, 512, kernel_size=(2, 2), stride=(2, 2))
      (1): LayerNorm()
      (2): GELU(approximate='none')
      (3): ConvTranspose2d(512, 256, kernel_size=(2, 2), stride=(2, 2))
      (4): Conv2d(
        256, 256, kernel_size=(1, 1), stride=(1, 1), bias=False
        (norm): LayerNorm()
      )
      (5): Conv2d(
        256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False
        (norm): LayerNorm()
      )
    )
    (simfp_3): Sequential(
      (0): ConvTranspose2d(1024, 512, kernel_size=(2, 2), stride=(2, 2))
      (1): Conv2d(
        512, 256, kernel_size=(1, 1), stride=(1, 1), bias=False
        (norm): LayerNorm()
      )
      (2): Conv2d(
        256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False
        (norm): LayerNorm()
      )
    )
    (simfp_4): Sequential(
      (0): Conv2d(
        1024, 256, kernel_size=(1, 1), stride=(1, 

In [4]:
torch_dtype=torch.float16
sd_pipe = AutoPipelineForText2Image.from_pretrained(
    path.join(model_path, sd_id), torch_dtype=torch_dtype).to(device)
explainer = StableDiffusionPipelineDetExplainer(pipe=sd_pipe, det_model=eva)

Loading pipeline components...:   0%|          | 0/7 [00:00<?, ?it/s]

/opt/conda/envs/snpa/lib/python3.8/site-packages/transformers/models/clip/feature_extraction_clip.py:28: FutureWarning: The class CLIPFeatureExtractor is deprecated and will be removed in version 5 of Transformers. Please use CLIPImageProcessor instead.
  warnings.warn(


In [5]:
raw_emb = torch.load('/mnt/workspace/data/tensor/raw_emb_tensor_cat_1.pt')
gen_prefix_embs = torch.load('/mnt/workspace/data/tensor/gen_prefix_embs_tensor_cat_1.pt')

gen_text_hidden_fcs = layers.GenTextHiddenFcs()
gill_state_dict = torch.load('/mnt/workspace/github/gill/checkpoints/gill_opt/pretrained_ckpt.pth.tar')

gen_text_hidden_fcs_state_dict = {}
for key, val in gill_state_dict['state_dict'].items():
    if 'gen_text_hidden_fcs' in key:
        prefix = 'gen_text_hidden_fcs' + key.split('gen_text_hidden_fcs')[1]
        gen_text_hidden_fcs_state_dict[prefix] = val
gen_text_hidden_fcs.load_state_dict(gen_text_hidden_fcs_state_dict)
gen_text_hidden_fcs.cuda()

/opt/conda/envs/snpa/lib/python3.8/site-packages/torch/nn/modules/transformer.py:306: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.norm_first was True
  warnings.warn(f"enable_nested_tensor is True, but self.use_nested_tensor is False because {why_not_sparsity_fast_path}")


GenTextHiddenFcs(
  (gen_text_hidden_fcs): ModuleList(
    (0): TextFcLayer(
      (fc): Linear(in_features=4096, out_features=512, bias=True)
      (tfm): Transformer(
        (encoder): TransformerEncoder(
          (layers): ModuleList(
            (0-3): 4 x TransformerEncoderLayer(
              (self_attn): MultiheadAttention(
                (out_proj): NonDynamicallyQuantizableLinear(in_features=512, out_features=512, bias=True)
              )
              (linear1): Linear(in_features=512, out_features=2048, bias=True)
              (dropout): Dropout(p=0.0, inplace=False)
              (linear2): Linear(in_features=2048, out_features=512, bias=True)
              (norm1): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
              (norm2): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
              (dropout1): Dropout(p=0.0, inplace=False)
              (dropout2): Dropout(p=0.0, inplace=False)
            )
          )
          (norm): LayerNorm((512,), ep

In [7]:
with amp.autocast(enabled=True):
    gen_emb = gen_text_hidden_fcs.gen_text_hidden_fcs[0](raw_emb, gen_prefix_embs)

    output = explainer(
        prompt_embeds=gen_emb,
        num_inference_steps=1,
        target_cls_id=15,
        raw_embeds=raw_emb
    )

  0%|          | 0/1 [00:00<?, ?it/s]

Calculating token attributions... 

/opt/conda/envs/snpa/lib/python3.8/site-packages/torch/nn/modules/conv.py:952: UserWarning: Plan failed with an OutOfMemoryError: CUDA out of memory. Tried to allocate 110.00 MiB. GPU  (Triggered internally at /opt/conda/conda-bld/pytorch_1712608851799/work/aten/src/ATen/native/cudnn/Conv_v8.cpp:924.)
  return F.conv_transpose2d(


OutOfMemoryError: CUDA out of memory. Tried to allocate 144.00 MiB. GPU 